In [1]:
from rake_nltk import Rake
import nltk
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import warnings
import string
warnings.filterwarnings("ignore")

In [2]:
df_csv = pd.read_csv('C:/Users/camil/Desktop/Projeto Integrador/dados/final.csv')

In [3]:
df_csv.head()

,Unnamed: 0,tconst,primaryTitle,startYear,runtimeMinutes,genre_1,genre_2,genre_3,Item,Mês,...,directors_2_films,directors_1_ratingAverage,directors_2_ratingAverage,writer_1_films,writer_2_films,writer_1_ratingAverage,writer_2_ratingAverage,AvgRating,numFilmes,num_crew
0,66398,tt0118589,Glitter,2001,104,Drama,Music,Romance,3947,Setembro,...,0,6.90,NaN,0,1,NaN,6.900000,5.800000,0.25,10
1,67605,tt0120681,From Hell,2001,122,Horror,Mystery,Thriller,1732,Outubro,...,3,7.31,7.31,0,0,NaN,NaN,6.258212,1.00,10
2,67642,tt0120737,The Lord of the Rings: The Fellowship of the Ring,2001,178,Action,Adventure,Drama,356,Dezembro,...,0,7.16,NaN,0,4,NaN,7.262979,6.533333,0.75,10
3,67655,tt0120755,Mission: Impossible II,2000,123,Action,Adventure,Thriller,297,Maio,...,0,7.11,NaN,3,0,7.099574,NaN,NaN,0.00,10
4,68298,tt0122459,Return to Me,2000,115,Comedy,Drama,Romance,2404,Abril,...,0,NaN,NaN,0,0,NaN,NaN,NaN,0.00,10


In [4]:
df_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1490 entries, 0 to 1489
Data columns (total 67 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Unnamed: 0                            1490 non-null   int64  
 1   tconst                                1490 non-null   object 
 2   primaryTitle                          1490 non-null   object 
 3   startYear                             1490 non-null   int64  
 4   runtimeMinutes                        1490 non-null   int64  
 5   genre_1                               1490 non-null   object 
 6   genre_2                               1373 non-null   object 
 7   genre_3                               997 non-null    object 
 8   Item                                  1490 non-null   int64  
 9   Mês                                   1490 non-null   object 
 10  month                                 1490 non-null   float64
 11  day              

In [5]:
# salvar o nome do filme e o roteiro num plot
data = {'primaryTitle':['Matrix 4'], 'movie_info':['The Matrix 4 is an upcoming American science fiction action film and the fourth installment in The Matrix franchise. The film is co-written and directed by Lana Wachowski, one of the two Wachowski sisters who directed the previous three films. Keanu Reeves, Carrie-Anne Moss, Jada Pinkett Smith, and Lambert Wilson reprise their roles from previous films in the series. The film will be a joint production by Warner Bros. Pictures and Village Roadshow Pictures and is scheduled to be released on April 1, 2022.']} 
  
# Create DataFrame 
dfx = pd.DataFrame(data) 

In [6]:
dfx

,primaryTitle,movie_info
0,Matrix 4,The Matrix 4 is an upcoming American science f...


In [7]:
# criar copias dos dois datasets para manipular os dados
df_copy = df_csv.copy()
df2 = dfx.copy()

# excluir linhas repetidas
df_copy.drop_duplicates(subset = 'movie_info', inplace = True)

# selecionar colunas de interesse dos dois datasets
df_copy = df_copy[['primaryTitle', 'movie_info']]


# excluir missing values
df_copy.dropna(inplace=True)

# juntar os dois datasets
df = df_copy.append(df2, ignore_index=True)

In [8]:
df[df.primaryTitle == 'Matrix 4']

,primaryTitle,movie_info
1480,Matrix 4,The Matrix 4 is an upcoming American science f...


In [9]:
# remoção da pontuação de 'Plot'
df['movie_info'] = df['movie_info'].str.replace('[{}]'.format(string.punctuation), '')

In [10]:
 # inicialização das lista que será a nova coluna
df['Key_words'] = ''  

# instanciação do objeto do Rake para descartar as 'stop words' (baseado naquilo que está no NLTK)
r = Rake()

for index, row in df.iterrows():
    # extração das key words - que já remove as 'stop words' do texto
    r.extract_keywords_from_text(row['movie_info'])   
    # dicionário com as key words e scores
    key_words_dict_scores = r.get_word_degrees()    
    # nova coluna que será inserida
    row['Key_words'] = list(key_words_dict_scores.keys())   

In [11]:
'''# to extract all genre into a list, only the first three actors into a list, and all directors into a list
df['genre_1'] = df['genre_1'].map(lambda x: x.split(','))
df['directors_1'] = df['directors_1'].map(lambda x: x.split(','))
#df['casts'] = df['casts'].map(lambda x: x.split(',')[:3])'''

"# to extract all genre into a list, only the first three actors into a list, and all directors into a list\ndf['genre_1'] = df['genre_1'].map(lambda x: x.split(','))\ndf['directors_1'] = df['directors_1'].map(lambda x: x.split(','))\n#df['casts'] = df['casts'].map(lambda x: x.split(',')[:3])"

In [12]:
'''# Retirada dos espaços dos campos e conversão para minúsculas
for index, row in df.iterrows():
    row['genre_1'] = [x.lower().replace(' ','') for x in row['genre_1']] 
    row['directors_1'] = [x.lower().replace(' ','') for x in row['directors_1']]
    #row['directors'] = [x.lower().replace(' ','') for x in row['directors']]'''

"# Retirada dos espaços dos campos e conversão para minúsculas\nfor index, row in df.iterrows():\n    row['genre_1'] = [x.lower().replace(' ','') for x in row['genre_1']] \n    row['directors_1'] = [x.lower().replace(' ','') for x in row['directors_1']]\n    #row['directors'] = [x.lower().replace(' ','') for x in row['directors']]"

In [13]:
# inicialização da lista
df['Bag_of_words'] = ''
# colunas que farão parte da Bag of Words
#columns = ['genres', 'directors', 'casts', 'Key_words']
columns = [ 'Key_words']
# percorre cada coluna e adiciona cada palavra individualmente
for index, row in df.iterrows():
    words = ''
    for col in columns:
        words += ' '.join(row[col]) + ' '
    row['Bag_of_words'] = words

In [14]:
# remoção dos "espaços" de todas as palavras na BoW
df['Bag_of_words'] = df['Bag_of_words'].str.strip().str.replace('   ', ' ').str.replace('  ', ' ')

In [15]:
df_bag = df[['primaryTitle','Bag_of_words']]

In [16]:
# Geração da Matriz com a frequência de cada palavra com filme 250 x quantidade total de palavras únicas
count = CountVectorizer()
count_matrix = count.fit_transform(df_bag['Bag_of_words'])

In [17]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [18]:
df_bag.reset_index(inplace = True)

In [19]:
# to create a Series for movie titles which can be used as indices (each index is mapped to a movie title)
indices = pd.Series(df_bag['primaryTitle'])

In [20]:
def recommend(title, cosine_sim = cosine_sim):
    # inicialização da lista
    recommended_movies = []
    # captura o índice que corresponde ao título
    idx = indices[indices == title].index[0]   
    # scores de similaridade em ordem descendente
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)   
    # captura os 10 filmes com o maior índice de similaridade
    top_10_indices = list(score_series.iloc[1:11].index)   
    # [1:11] para excluir o primeiro (0) que é o próprio filme selecionado
    
    # monta uma lista com os filmes retornados
    for i in top_10_indices:   
        recommended_movies.append(list(df_bag['primaryTitle'])[i])
        
    return recommended_movies

In [48]:
#filme_simple = recommend("The Lord of the Rings: The Fellowship of the Ring")
filme = recommend("Matrix 4")
# recomendacao para o filme que sera produzido
#filme = recommend(df2.movie_name)

In [32]:
filme

['Paranormal Activity 4',
 'The Matrix Revolutions',
 'Highlander: Endgame',
 'Annabelle',
 'The Final Destination',
 "Bridget Jones's Baby",
 'Dead Man Down',
 'The Conjuring 2',
 'Horrible Bosses 2',
 'Hall Pass']

In [24]:
df_rec = df_csv[df_csv.primaryTitle.isin(filme)]

In [25]:
df_rec.head()

,Unnamed: 0,tconst,primaryTitle,startYear,runtimeMinutes,genre_1,genre_2,genre_3,Item,Mês,...,directors_2_films,directors_1_ratingAverage,directors_2_ratingAverage,writer_1_films,writer_2_films,writer_1_ratingAverage,writer_2_ratingAverage,AvgRating,numFilmes,num_crew
12,74357,tt0144964,Highlander: Endgame,2000,87,Action,Adventure,Fantasy,3168,Setembro,...,0,NaN,NaN,5,2,6.327132,5.764414,NaN,0.00,10
139,96723,tt0242653,The Matrix Revolutions,2003,129,Action,Sci-Fi,NaN,188,Novembro,...,2,8.66,8.66,3,3,8.555799,8.555799,6.775322,3.25,10
621,136040,tt0480687,Hall Pass,2011,105,Comedy,Romance,NaN,780,Fevereiro,...,9,6.74,6.74,2,5,6.478528,6.648043,6.537045,8.25,10
829,160074,tt1144884,The Final Destination,2009,82,Horror,Thriller,NaN,1456,Agosto,...,0,5.97,NaN,3,1,7.248064,5.100000,6.150624,1.25,10
979,177935,tt1473832,Bridget Jones's Baby,2016,123,Comedy,Drama,Romance,1747,Setembro,...,0,6.67,NaN,2,2,6.476391,6.050723,5.945229,11.50,10


### Análise de sentimento

In [28]:
print(float(df_csv.negative_perc[df_csv.primaryTitle.isin(filme)].mean()))
print(float(df_csv.negative_perc[df_csv.primaryTitle.isin(filme)].min()))
print(float(df_csv.negative_perc[df_csv.primaryTitle.isin(filme)].max()))

42.964482834735385
23.07692307692308
59.18367346938776


In [39]:
# salvar essas informacoes como novas colunas do dataset 
dfx['negative_perc_avg'] = float(df_csv.negative_perc[df_csv.primaryTitle.isin(filme)].mean())
dfx['negative_perc_min'] = float(df_csv.negative_perc[df_csv.primaryTitle.isin(filme)].min())
dfx['negative_perc_max'] = float(df_csv.negative_perc[df_csv.primaryTitle.isin(filme)].max())

In [40]:
print(float(df_csv.positive_perc[df_csv.primaryTitle.isin(filme)].mean()))
print(float(df_csv.positive_perc[df_csv.primaryTitle.isin(filme)].min()))
print(float(df_csv.positive_perc[df_csv.primaryTitle.isin(filme)].max()))

57.035517165264615
40.81632653061224
76.92307692307693


In [41]:
# salvar essas informacoes como novas colunas do dataset 
dfx['positive_perc_avg'] = float(df_csv.negative_perc[df_csv.primaryTitle.isin(filme)].mean())
dfx['positive_perc_min'] = float(df_csv.negative_perc[df_csv.primaryTitle.isin(filme)].min())
dfx['positive_perc_max'] = float(df_csv.negative_perc[df_csv.primaryTitle.isin(filme)].max())

### Número de votos

In [30]:
print(float(df_csv.numVotes[df_csv.primaryTitle.isin(filme)].mean()))
print(float(df_csv.numVotes[df_csv.primaryTitle.isin(filme)].min()))
print(float(df_csv.numVotes[df_csv.primaryTitle.isin(filme)].max()))

135550.5
18990.0
434707.0


In [42]:
# salvar essas informacoes como novas colunas do dataset 
dfx['numVotes_avg'] = float(df_csv.numVotes[df_csv.primaryTitle.isin(filme)].mean())
dfx['numVotes_min'] = float(df_csv.numVotes[df_csv.primaryTitle.isin(filme)].min())
dfx['numVotes_max'] = float(df_csv.numVotes[df_csv.primaryTitle.isin(filme)].max())

### Média das Ratings

In [31]:
print(float(df_csv.AvgRating[df_csv.primaryTitle.isin(filme)].mean()))
print(float(df_csv.AvgRating[df_csv.primaryTitle.isin(filme)].min()))
print(float(df_csv.AvgRating[df_csv.primaryTitle.isin(filme)].max()))

6.311817222476072
5.219213301277744
6.978405117734216


In [43]:
# salvar essas informacoes como novas colunas do dataset 
dfx['AvgRating_avg'] = float(df_csv.AvgRating[df_csv.primaryTitle.isin(filme)].mean())
dfx['AvgRating_min'] = float(df_csv.AvgRating[df_csv.primaryTitle.isin(filme)].min())
dfx['AvgRating_max'] = float(df_csv.AvgRating[df_csv.primaryTitle.isin(filme)].max())

In [44]:
dfx

,primaryTitle,movie_info,negative_perc,positive_perc,negative_perc_avg,negative_perc_min,negative_perc_max,positive_perc_avg,positive_perc_min,positive_perc_max,numVotes_avg,numVotes_min,numVotes_max,AvgRating_avg,AvgRating_min,AvgRating_max
0,Matrix 4,The Matrix 4 is an upcoming American science f...,59.183673,59.183673,42.964483,23.076923,59.183673,42.964483,23.076923,59.183673,135550.5,18990.0,434707.0,6.311817,5.219213,6.978405
